## CS431/631 Data Intensive Distributed Computing
### Fall 2021 - Assignment 4
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Ze Sheng
* **ID:** 20848004

Unlike our previous environment, Spark is not installed in Colab so we have to install it ourself. This will take a minute to finish.

In [1]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Next we need to create SparkContext. Run the following block to set the environment path and create a SparkContext.

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(appName="YourTest", master="local[*]")

---
#### Overview
For this assignment, you will be using Python and Spark to perform spam detection.   You will need to perform two tasks.   The first is to build spam prediction models, using training data sets and stochastic gradient descent (SGD).   The second is to use these models to predict whether the documents in a test data set are spam.
The stochastic gradient descent technique that you will be using is based on [a paper](http://arxiv.org/abs/1004.5168) by Cormack, Smucker and Clarke.

#### Training a Spam Classification Models
To build a spam classification model, you will start with a training data set.   Each instance in the training set represents a single document, and is labeled to indicate whether that document should be considered to be spam or ham.
An instance looks like this:
```
clueweb09-en0094-20-13546 spam 387908 697162 426572 161118 688171 ...
```
The first field, `clueweb09-en0094-20-13546`, is the (unique) document name.   The second field is the label, indicating whether the document should be considered spam (as in this example) or ham.   The remaining fields are integers representing *features* present in the document.   In this case, the features are hashed byte 4-grams, represented as integers.   Each training data set is stored as a text file, with one instance per line.   The training files  are:
* `spam.train.group_x.txt`   (25 MB)
* `spam.train.group_y.txt`   (20 MB)
* `spam.train.britney.txt`   (766 MB)

Now let's download the spamminess module and the training traces we will use in this assignment. This will take a few minutes. The ls command at the end shows the files we have in this directory. Make sure all files are here now.

In [3]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/W20/content/cs431/spamminess.py
!wget -q https://www.student.cs.uwaterloo.ca/~cs451/spam/spam.train.group_x.txt.bz2
!wget -q https://www.student.cs.uwaterloo.ca/~cs451/spam/spam.train.group_y.txt.bz2
!wget -q https://www.student.cs.uwaterloo.ca/~cs451/spam/spam.train.britney.txt.bz2

!bunzip2 spam.train.group_x.txt.bz2
!bunzip2 spam.train.group_y.txt.bz2
!bunzip2 spam.train.britney.txt.bz2
!ls

bunzip2: Output file spam.train.group_x.txt already exists.
bunzip2: Output file spam.train.group_y.txt already exists.
bunzip2: Output file spam.train.britney.txt already exists.
compute_spam_metrics	  spam.test.qrels.txt.bz2
compute_spam_metrics.c	  spam.train.britney.txt
compute_spam_metrics.c.1  spam.train.britney.txt.bz2
models			  spam.train.group_x.txt
__pycache__		  spam.train.group_x.txt.bz2
results			  spam.train.group_y.txt
sample_data		  spam.train.group_y.txt.bz2
spam_eval.sh		  spark-2.4.8-bin-hadoop2.7
spam_eval.sh.1		  spark-2.4.8-bin-hadoop2.7.tgz
spamminess.py		  spark-2.4.8-bin-hadoop2.7.tgz.1
spamminess.py.1		  tmp
spam.test.qrels.txt



---
### Important

The questions that follow ask you to implement functions whose prototypes are given to you. Do _**not**_ change the prototypes of the functions. Do _**not**_ write code outside of the functions.

You may use specific cells, identified by `# Your tests here`, for test purposes. Code in these cells will *not* be executed when marking your assignment.

---

#### Question 1 ( 5/20 marks)

Your first task is to write a sequential SGD model trainer in Python (no Spark).   For our purposes, a model associates a *weight* with each feature.   The model trainer decides what these weights should be, based on the training instances.  Since you are going to be writing a model trainer based on SGD, the trainer should behave like this:
```
for each training instance T
   predict whether T is spam or ham using the weights of the current model
   update the model weights by comparing T's predicted label with its actual label
```
Of course, the important part is how to update the model.

In [the paper](http://arxiv.org/abs/1004.5168), the model is used to assign a "spamminess" score to a document.   Documents with positive spamminess are predicted to be spam.   Those with negative spamminess are predicted to be ham.  The spamminess of a document $D$ is simply the sum of the weights (from the model) of each of the document's features:
\begin{equation}
spamminess(D) = \sum_{f \in D}{w(f)}
\end{equation}
where $w(f)$ is the weight assocated with feature $f$.

The Python module `spamminess.py` defines a function `spamminess(F,W)` which computes this quantity.   This function takes two arguments, `F` and `W`.  `F` is a list of features (integers) associated to the document whose spamminess you want to compute, and `W` is a dictionary representing the current model.  `W` maps features ($f$) to their weights ($w(f)$) under the model.

In the cell below, you will find partial pseudo-code that shows how to implement the SGD model trainer defined by Cormack, Smucker, and Clarke.   It reads the training instances one at a time from one of the training files, and uses them to adjust the model weights.   Your job is to turn this pseudo-code into actual runnable Python code that can
be used to learn a model from any one of the training files. Implement the function `sequential_SGD()` that takes as input a model (`w`), the training dataset and a value for the update parameter `delta`, and returns the trained model.

In [4]:
from spamminess import spamminess
from math import exp

def sequential_SGD(model, training_dataset='spam.train.group_x.txt', delta = 0.002):
    #### Your solution to Question 1 here
    # open one of the training files - defaults to group_x
    with open(training_dataset) as f:
        for i, line in enumerate(f):
            name, target, suffix = line.split(maxsplit=2)
            features = suffix.split()
            score = spamminess(features, model)
            prob = 1.0 / (1 + exp(-score))
            update = (1.0 - prob) * delta if target == "spam" else (-prob) * delta
            for f in features:
                model[f] = model.get(f, 0) + update
    
    return model

    #   for line in f:
    #      each line represents a document
    #      read and parse the line
    #      Let:
    #        t represent the spam/ham tag for this document
    #        F represent the list of features for this document

    #      find the spamminess of the current document using the current model:
    #      score = spamminess(F,w)

    #      then, update the model:
    #      prob = 1.0/(1+exp(-score))
    #      for each feature f in F:
    #          if t == 'spam':
    #              increase model(f) by (1.0-prob)*delta (or set model(f) to (1.0-prob)*delta if f is not in the dict yet)
    #          elif t == 'ham':
    #              decrease model(f) by prob*delta (or set model(f) to -prob*delta if f is not in the dict yet)
    # return model


In [5]:
# Your tests here
# Describe the data set
flag = True
fMax = 0
# with open('spam.train.group_x.txt') as fh:
with open('spam.train.britney.txt') as fh:
    for i, line in enumerate(fh):
        flag &= len(line) > 10
        name, target, suffix = line.split(maxsplit=2)
        fMax = max(fMax, len(suffix.split()))
    
print(flag)
print(i)
print(fMax)

True
21367
23558


In [6]:
# Your tests here
ks = sequential_SGD({}, training_dataset="spam.train.britney.txt").keys()
print(min(ks))
print(max(ks))
print(len(ks))

1
999999
966064


In [7]:
# Your tests here
w = sequential_SGD({}) # Providing an empty model

#### Question 2 (5/20 marks)

Next, you should try implementing a Spark version of the SGD model trainer.   Your Spark implementation should read a training file, train the model, and then output the model to the `models` folder.  The model output file that you generate should list the weight associated with each feature, with one feature per line, like this:
```
(802123, 0.0009858585991850937)
(438450, 4.267897922108138e-05)
(271525, 0.0013133437007968654)
(92853, 0.0004300009932503611)
```

Use Spark's `saveAsTextFile` action to output your model.   For example, if you are training a model for the group_x training set, use `saveAsTextFile("models/group_x_model")`.   This will actually cause Spark to create a folder called `group_x_model`.   In the folder, there will be files with names like `part-00000` that contain the actual output data.  When you use `saveAsTextFile`, Spark will generate one `part-xxxxx` file for each partition of the RDD that you are writing out.   In this case, you should have only a single partition (for the reason described below), so there should be only one `part-xxxxx` file.

Training the SGD model is an inherently sequential task, since the training instances update the model one at a time, and each instance's spamminess is computed using the model produced by that instance's predecessors.   This means that the only part of the training that you can parallelize using Spark is the parsing of the input file.   Once the input is parsed, your Spark implementation will have to force all of the instances into a single partition, and then apply the training function to the entire partition.   To see whether you are getting sensible results, you can compare the model you learn with Spark to the one that you learned with your sequential Python program from Question 1.

Remember that training should occur entirely in Spark.  The training instances should never come into your driver program.

Implement the function `spark_SGD()` below that takes as input the path to the training dataset, an output path `output_model` and a value for the update parameter `delta`, and writes the trained model to `output_model` using Spark's `saveAsTextFile`. You can use it to generate models from all three of the training files, leaving the results in your models folder. For this assignment, you will be using Spark's original RDD interface, rather than the DataFrame interface.

Hint: You need to move all of the data into one partition and then use mapPartition to train the model.


In [8]:
from spamminess import spamminess
from math import exp
import shutil, os

def spark_SGD(training_dataset='spam.train.group_x.txt', output_model='models/group_x_model', delta = 0.002):
    if os.path.isdir(output_model):
        shutil.rmtree(output_model) # Remove the previous model to create a new one
    training_data = sc.textFile(training_dataset)
    #### Your Solution to Question 2 here

    def _init_mapper(line):
        name, target, suffix = line.split(maxsplit=2)
        features = list(map(int, suffix.split()))
        return (target == "spam", features)
    
    def _train_mapper(lines):
        model = {}
        for label, features in lines:
            score = spamminess(features, model)
            prob = 1.0 / (1 + exp(-score))
            update = (1.0 - prob) * delta if label else (-prob) * delta
            for f in features:
                model[f] = model.get(f, 0) + update
        return map(lambda x: (str(x[0]), x[1]), model.items())
    
    model = sc.textFile(training_dataset).map(_init_mapper).coalesce(1).mapPartitions(_train_mapper)
    model.saveAsTextFile(output_model)

    return model.top(10, lambda x: x[1])



In [9]:
# Your tests here
top_x = spark_SGD()
top_y = spark_SGD(training_dataset='spam.train.group_y.txt', output_model='models/group_y_model')
top_b = spark_SGD(training_dataset='spam.train.britney.txt', output_model='models/britney_model')

print(top_x, "\n")
print(top_y, "\n")
print(top_b, "\n")

!ls -lh /content/models/group_x_model/
!head -n 5 /content/models/group_x_model/part-00000
print()
!ls -lh /content/models/group_y_model/
!head -n 5 /content/models/group_y_model/part-00000
print()
!ls -lh /content/models/britney_model/
!head -n 5 /content/models/britney_model/part-00000

[('288281', 0.022996007768337472), ('316070', 0.02178760768974702), ('305568', 0.02166991395934579), ('737304', 0.02155627086676939), ('102264', 0.021381723804112546), ('202131', 0.020560097038631847), ('674600', 0.02024316823828206), ('419514', 0.0200693349931559), ('837735', 0.019975296559266612), ('879817', 0.01985728645880699)] 

[('718475', 0.019444116877822448), ('111709', 0.019160666157830485), ('134614', 0.017896744467459636), ('356301', 0.01713973961504013), ('189211', 0.016890536439803084), ('46415', 0.01683395403857716), ('699656', 0.016717887604709453), ('720009', 0.01644869972017484), ('278753', 0.016435449365613492), ('109474', 0.016277191144830425)] 

[('953797', 0.21247124013078147), ('143843', 0.2123672246173025), ('791887', 0.2071148494609864), ('440566', 0.19685927612839674), ('498211', 0.09808175979511968), ('372064', 0.08912006930282125), ('754151', 0.0813435307182448), ('441602', 0.08075078611314168), ('111709', 0.07930317843081394), ('332134', 0.0732230874402228)

#### Question 3 (5/20 marks)

When you train a model using SGD, the model you get depends on the order in which you handle the training instances.  To see this in action, try using the Spark SGD trainer you implemented for Question 2 to train a model from the group_x training set, but with the instances processed in a different order.  

To do this, re-implement your trainer from Question 2 so that it will randomly reorder the training instances before using them to update the model. One way to shuffle the training instances is to assign a random sort key to each training instance as you read it from the input file, and then sort the instances using the random sort key.

Be sure that Spark is doing the work of shuffling the training instances.   Do not load the training instances into your driver program and sort them there.

Implement the function `spark_shuffled_SGD` below that takes as input the path to the training dataset, an output path `output_model` and a value for the update parameter `delta`, shuffles the training instances using the method described above and writes the trained model to `output_model` using Spark's `saveAsTextFile`.

Once you have implemented the shuffled trainer, train a model using shuffled group_x training instances, and compare the resulting model with group_x model you learned without shuffling.  It is up to you how to do this comparision.  At a minimum, compare features with the highest weights in each model to see if they are similar. You can also use the classifier in next question to classify documents using the two models, and compare results.


In [10]:
from spamminess import spamminess
from math import exp
import shutil, os, random

def spark_shuffled_SGD(training_dataset='spam.train.group_x.txt', output_model='models/group_x_model', delta = 0.002):
    if os.path.isdir(output_model):
        shutil.rmtree(output_model) # Remove the previous model to create a new one
    training_data = sc.textFile(training_dataset)
    #### Your Solution to Question 3 here

    def _init_mapper(line):
        name, target, suffix = line.split(maxsplit=2)
        features = list(map(int, suffix.split()))
        return (target == "spam", features, random.randrange(1 << 30))
    
    def _train_mapper(lines):
        model = {}
        for label, features, _ in lines:
            score = spamminess(features, model)
            prob = 1.0 / (1 + exp(-score))
            update = (1.0 - prob) * delta if label else (-prob) * delta
            for f in features:
                model[f] = model.get(f, 0) + update
        return map(lambda x: (str(x[0]), x[1]), model.items())
    

    model = sc.textFile(training_dataset).map(_init_mapper).coalesce(1).sortBy(lambda x: x[2]).mapPartitions(_train_mapper)
    model.saveAsTextFile(output_model)

    return model.top(10, lambda x: x[1])



In [11]:
# Your tests here
top_x = spark_shuffled_SGD(output_model='models/group_x_model_shuffled')
top_y = spark_shuffled_SGD(output_model='models/group_y_model_shuffled')
top_b = spark_shuffled_SGD(output_model='models/britney_model_shuffled')

# The top-10 largest parameters are not similar to that of non-shuffled SGD. 
print(top_x, "\n")
print(top_y, "\n")
print(top_b, "\n")

!ls -lh /content/models/group_x_model_shuffled
!head -n 5 /content/models/group_x_model_shuffled/part-00000
print()
!ls -lh /content/models/group_y_model_shuffled
!head -n 5 /content/models/group_y_model_shuffled/part-00000
print()
!ls -lh /content/models/britney_model_shuffled
!head -n 5 /content/models/britney_model_shuffled/part-00000

[('658098', 0.02597478650303097), ('293364', 0.02597478650303097), ('756969', 0.02523687738633835), ('305568', 0.0241927717147492), ('379514', 0.02261478039660749), ('202131', 0.02218302172505044), ('111724', 0.021168396815838063), ('822753', 0.021071398233594923), ('837735', 0.020527584442425497), ('749735', 0.020199023127675648)] 

[('658098', 0.02679475493492942), ('293364', 0.02679475493492942), ('288281', 0.023823542453923593), ('379514', 0.022931020432710926), ('756969', 0.02277686301384611), ('647380', 0.022536462351052104), ('364959', 0.02166856161999796), ('305568', 0.021063316283433674), ('73896', 0.02093289006021485), ('288900', 0.02071547093214718)] 

[('401471', 0.02441769687064352), ('658098', 0.023367631403392274), ('293364', 0.023367631403392274), ('91809', 0.022737261995313396), ('288281', 0.022701690393592246), ('520474', 0.022458661508337883), ('837735', 0.02229548963909115), ('278969', 0.021731064506228002), ('850147', 0.0215134446089048), ('816105', 0.0214537024836

#### Question 4 (5/20  marks)

Last but not least, you should write a Spark program that can be used to classify documents as spam or ham, using the classification models you produced.

The test data, i.e., the document instances that you should classifiy, are located in `spam.test.qrels.txt`. Run the following block to download this trace. This will take a few minutes.

In [12]:
!wget -q https://www.student.cs.uwaterloo.ca/~cs451/spam/spam.test.qrels.txt.bz2
!bunzip2 spam.test.qrels.txt.bz2
!ls

bunzip2: Output file spam.test.qrels.txt already exists.
compute_spam_metrics	  spam.test.qrels.txt.bz2
compute_spam_metrics.c	  spam.test.qrels.txt.bz2.1
compute_spam_metrics.c.1  spam.train.britney.txt
models			  spam.train.britney.txt.bz2
__pycache__		  spam.train.group_x.txt
results			  spam.train.group_x.txt.bz2
sample_data		  spam.train.group_y.txt
spam_eval.sh		  spam.train.group_y.txt.bz2
spam_eval.sh.1		  spark-2.4.8-bin-hadoop2.7
spamminess.py		  spark-2.4.8-bin-hadoop2.7.tgz
spamminess.py.1		  spark-2.4.8-bin-hadoop2.7.tgz.1
spam.test.qrels.txt	  tmp



Each line in this file represents a document that needs to be classified as spam or ham.  The format of this file is identical to the format of the files that hold the training instances.

Implement the function `spark_classify` below that will load a model (from a specified folder under `models`), classify all of the instances in a given test data file (`spam.test.qrels.txt` by default) using that model, and then output the results in the folder `results_path` using Spark's `saveAsTextFile`.   The contents of the output file should look like this:
```
(clueweb09-en0000-00-00142,spam,2.601624279252943,spam)
(clueweb09-en0000-00-01005,ham,2.5654162439491004,spam)
(clueweb09-en0000-00-01382,ham,2.5893946346394188,spam)
```
Each line of the output represents one test instance.   The first two fields are the document ID and the test label.  These are just copied from the test data.   The third field is the spamminess score of the document, produced by the spamminess function using the model you are classifying with.   The fourth field is the spam/ham prediction made by the model.

Of course, your spam/ham classifier must **not** use the test label from the input when making its prediction.  The test labels are the "ground truth" against which your predictions are being compared.   Using them to make predictions would defeat the whole purpose of model-based classification.

Make sure that classification of the test instances is done by Spark, not by your driver program.  Do ***not*** load the test instances or classification results into your driver program. You are however allowed to load the model weights into your driver program to distribute them as side data. 
Unlike model training, classification is easily parallelizable, since each document is classified independently. 

In [15]:
from spamminess import spamminess
import shutil, os

def spark_classify(input_model='models/group_x_model', test_dataset='spam.test.qrels.txt', results_path='results/test_qrels'):
    if os.path.isdir(results_path):
        shutil.rmtree(results_path) # Remove the previous results
    test_data = sc.textFile(test_dataset)
    #### Your Solution to Question 4 here

    def _model_mapper(line):
        f, param = line[2:-1].split("', ")
        return (int(f), float(param))
    
    def _predict_mapper(line):
        name, target, suffix = line.split(maxsplit=2)
        features = list(map(int, suffix.split()))
        score = spamminess(features, model)
        pred = "spam" if score > 0 else "ham"
        return (name, target, score, pred)
    

    model = sc.broadcast(sc.textFile(input_model).map(_model_mapper).collectAsMap()).value
    predictions = sc.textFile(test_dataset).map(_predict_mapper)
    predictions.saveAsTextFile(results_path)



In [16]:
# Your tests here
spark_classify(input_model='models/group_x_model', test_dataset='spam.test.qrels.txt', results_path='results/test_qrels_group_x')
spark_classify(input_model='models/group_y_model', test_dataset='spam.test.qrels.txt', results_path='results/test_qrels_group_y')
spark_classify(input_model='models/britney_model', test_dataset='spam.test.qrels.txt', results_path='results/test_qrels_britney')

In [17]:
# Your tests here
!ls results/test_qrels_group_x
!head -n 5 results/test_qrels_group_x/part-00000
print()
!ls results/test_qrels_group_y
!head -n 5 results/test_qrels_group_y/part-00000
print()
!ls results/test_qrels_britney
!head -n 20 results/test_qrels_britney/part-00000

part-00000  part-00006	part-00012  part-00018	part-00024  part-00030
part-00001  part-00007	part-00013  part-00019	part-00025  part-00031
part-00002  part-00008	part-00014  part-00020	part-00026  part-00032
part-00003  part-00009	part-00015  part-00021	part-00027  part-00033
part-00004  part-00010	part-00016  part-00022	part-00028  _SUCCESS
part-00005  part-00011	part-00017  part-00023	part-00029
('clueweb09-en0000-00-00142', 'spam', 2.601624279252943, 'spam')
('clueweb09-en0000-00-01005', 'ham', 2.5654162439491004, 'spam')
('clueweb09-en0000-00-01382', 'ham', 2.5893946346394188, 'spam')
('clueweb09-en0000-00-01383', 'ham', 2.6190102258752614, 'spam')
('clueweb09-en0000-00-03449', 'ham', 1.500142758578532, 'spam')

part-00000  part-00006	part-00012  part-00018	part-00024  part-00030
part-00001  part-00007	part-00013  part-00019	part-00025  part-00031
part-00002  part-00008	part-00014  part-00020	part-00026  part-00032
part-00003  part-00009	part-00015  part-00021	part-00027  part-00033

We have developed a program that can be used to evaluate your classification results.  Run the next block to download this program.

In [18]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/compute_spam_metrics.c
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/spam_eval.sh

Now compile this program.

In [19]:
!gcc -w -O2 -o compute_spam_metrics compute_spam_metrics.c -lm

 Given your ouput file, in the proper format, it will compute the area under the receiver operating curve (ROC).   This is a common way to characterize classifier error.    The lower this score, the better.   The evaluation program should produce one line of output, like this
```
1-ROCA%: 17.25
```

Use your classifier to classify the test instances using each of the three classification models that you produced, which should result in three different output files.   Then, in the cell below,
use the evaluation program to evaluate your results.


In [20]:
# Your tests here
#  Run the evaluation program like this, after first replacing "output-file"
#  with the name of the folder that holds your classifier's output
!bash spam_eval.sh results/test_qrels_group_x
!bash spam_eval.sh results/test_qrels_group_y
!bash spam_eval.sh results/test_qrels_britney

1-ROCA%: 17.26
1-ROCA%: 12.82
1-ROCA%: 15.95


---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.